In [1]:
import pickle, pandas as pd, numpy as np, re

In [2]:
entdicf = "entdic3hops.pkl"
reldicf = "reldic3hops.pkl"
triplesf = "triples3hops.pd.pkl"
allrelsf = "allrel3hops.pd.pkl"
typesf = "../dbpedia/dbpcore.nt.types.dok"
bigentdicf = "../dbpedia/dbpcore.nt.entdic.ssd"
rhsdicf = "rhsdic.pkl"
inclents = True

In [3]:
entdic = pickle.load(open(entdicf))
reldic = pickle.load(open(reldicf))
triples = pickle.load(open(triplesf))
allrels = pickle.load(open(allrelsf))
reventdic = {v: k for k, v in entdic.items()}
# load dictionaries
def loadDic(entdicf):
    entdic = {}
    for line in open(entdicf):
        splits = line.split(" ")
        entdic[splits[0]] = int(splits[1])
    return entdic

# load big dictionaries
bigentdic = loadDic(bigentdicf)
print "big entity dict loaded"
'''bigreldic = loadDic(bigreldicf)
print "big relation dict loaded"'''

big entity dict loaded


'bigreldic = loadDic(bigreldicf)\nprint "big relation dict loaded"'

In [4]:
reventdic = dict([(v, k) for k, v in entdic.items()])
revreldic = dict({(v, k) for k, v in reldic.items()})

# Triples

In [6]:
len(set(triples["s"]).union(set(triples["o"]))) # number of unique entities

11298

In [7]:
transentdic = dict(zip(set(triples["s"]).union(set(triples["o"])), range(len(set(triples["s"]).union(set(triples["o"]))))))
transreldic = dict(zip(set(triples["r"]), map(lambda x: x+len(transentdic), range(len(set(triples["r"]))))))

In [8]:
revtransentdic = {v: k for k, v in transentdic.items()}
revtransreldic = {v: k for k, v in transreldic.items()}

In [9]:
print set(revtransentdic.keys()).intersection(set(revtransreldic.keys())) # should be empty

set([])


In [10]:
revnewentdic = {k: reventdic[v] for k, v in revtransentdic.items()}
revnewreldic = {k: revreldic[v] for k, v in revtransreldic.items()}
invoffset = len(revnewreldic.keys())
revnewinvreldic = {k+invoffset: "-"+v for k, v in revnewreldic.items()}
revnewreldic.update(revnewinvreldic)
revnewdic = revnewentdic.copy()
revnewdic.update(revnewreldic)
newdic = {v: k for k, v in revnewdic.items()}

In [11]:
newdic['-http://dbpedia.org/ontology/location']

11310

In [13]:
newtriples = pd.DataFrame([triples["s"].map(lambda x: transentdic[x]), triples["r"].map(lambda x: transreldic[x]), triples["o"].map(lambda x: transentdic[x])])
newtriples = newtriples.T

In [14]:
#newtriples

In [15]:
newinvtriples = pd.DataFrame([newtriples["o"], newtriples["r"].map(lambda x: x+invoffset), newtriples["s"]]).T

In [16]:
newinvtriples.columns = ["s","r","o"]

In [19]:
# some dictionary checks
newtriples.apply(lambda x: x.map(lambda y: revnewdic[y]))
triples.apply(lambda x: x.map(lambda y: reventdic[y] if x.name != "r" else revreldic[y]))
pass

In [29]:
# reverse triples and include
alltriples = pd.concat([newtriples, newinvtriples]).reset_index(drop=True)
#alltriples

In [21]:
# saving triple tensor:
pickle.dump(newdic, open("tripletensor.flatidx.pkl","w"))
with open("tripletensor.flat.ssd", "w") as f:
    for _, row in newtriples.iterrows():
        f.write("%d %d %d 1.000\n" % (row["s"], row["r"], row["o"]))
with open("alltripletensor.flat.ssd", "w") as f:
    for _, row in alltriples.iterrows():
        f.write("%d %d %d 1.000\n" % (row["s"], row["r"], row["o"]))

## Train/Test

In [26]:
# split alltriples edges to train and test edges
import math
split = 0.1
splitidxs = range(alltriples.shape[0])
np.random.shuffle(splitidxs)
splitpoint = int(math.ceil(alltriples.shape[0] * (1 - split)))
alltriples_train = alltriples.iloc[splitidxs[:splitpoint]].sort_index()
alltriples_test = alltriples.iloc[splitidxs[splitpoint:]].sort_index()

In [27]:
# save
with open("alltripletensor.train.flat.ssd", "w") as f:
    for _, row in alltriples_train.iterrows():
        f.write("%d %d %d 1.000\n" % (row["s"], row["r"], row["o"]))
with open("alltripletensor.test.flat.ssd", "w") as f:
    for _, row in alltriples_test.iterrows():
        f.write("%d %d %d 1.000\n" % (row["s"], row["r"], row["o"]))

# Compat

In [40]:
allcompat = alltriples[["s", "r"]].drop_duplicates().reset_index(drop=True)

In [45]:
reloffset = min(set(allcompat["r"]))
reloffset

11298

In [50]:
# reindex
compatreldic = dict(map(lambda x: (revnewdic[x], x-reloffset), set(allcompat["r"])))
allcompat["r"] = allcompat["r"] - reloffset

In [52]:
allcompat_train = alltriples_train[["s", "r"]].drop_duplicates().reset_index(drop=True)
allcompat_test =  alltriples_test[["s", "r"]].drop_duplicates().reset_index(drop=True)
allcompat_train["r"] = allcompat_train["r"] - reloffset
allcompat_test["r"] = allcompat_test["r"] - reloffset

In [54]:
# save
pickle.dump(compatreldic, open("compatreldic.flatidx.pkl","w"))
with open("allcompat.flat.ssd", "w") as f:
    for _, row in allcompat.iterrows():
        f.write("%d %d 1.000\n" % (row["s"], row["r"]))
with open("allcompat.train.flat.ssd", "w") as f:
    for _, row in allcompat_train.iterrows():
        f.write("%d %d 1.000\n" % (row["s"], row["r"]))
with open("allcompat.test.flat.ssd", "w") as f:
    for _, row in allcompat_test.iterrows():
        f.write("%d %d 1.000\n" % (row["s"], row["r"]))